In [5]:
from langchain.utilities import SQLDatabase
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import create_sql_query_chain
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI

In [7]:
!pip install pymysql

In [13]:
#connect to the SQL database
host='localhost'
port='3306'
username='root'
password='1234567kid'
database_schema='HOCSINhDB'

mysql_uri = f'mysql+pymysql://{username}:{password}@{host}:{port}/{database_schema}'
db=SQLDatabase.from_uri(
    mysql_uri, 
    sample_rows_in_table_info=2
)

db.get_table_info()


c:\ProgramData\anaconda3\envs\medicalbot\lib\site-packages\langchain_community\utilities\sql_database.py:338: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "bomon, giaovien", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  metadata_table_names = [tbl.name for tbl in self._metadata.sorted_tables]
c:\ProgramData\anaconda3\envs\medicalbot\lib\site-packages\langchain_community\utilities\sql_database.py:350: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "bomon, giaovien", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  for tbl in self._metadata.sorted_tables


'\nCREATE TABLE bomon (\n\t`MABM` VARCHAR(5) NOT NULL, \n\t`TENBM` VARCHAR(50) CHARACTER SET utf8mb3 COLLATE utf8mb3_general_ci, \n\t`PHONG` VARCHAR(10) CHARACTER SET utf8mb3 COLLATE utf8mb3_general_ci, \n\t`DIENTHOAI` VARCHAR(15), \n\t`TRUONGBM` VARCHAR(5), \n\t`MAKHOA` VARCHAR(5), \n\t`NGAYNHANCHUC` DATE, \n\tPRIMARY KEY (`MABM`), \n\tCONSTRAINT `FK_BM_TRUONGBM` FOREIGN KEY(`TRUONGBM`) REFERENCES giaovien (`MAGV`)\n)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci\n\n/*\n2 rows from bomon table:\nMABM\tTENBM\tPHONG\tDIENTHOAI\tTRUONGBM\tMAKHOA\tNGAYNHANCHUC\nCNTT\tCông nghệ tri thức\tB15\t0838126126\tNone\tCNTT\tNone\nHHC\tHóa hữu cơ\tB44\t838222222\tNone\tHH\tNone\n*/\n\n\nCREATE TABLE chude (\n\t`MACD` CHAR(4) NOT NULL, \n\t`TENCD` VARCHAR(40) CHARACTER SET utf8mb3 COLLATE utf8mb3_general_ci, \n\tPRIMARY KEY (`MACD`)\n)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci\n\n/*\n2 rows from chude table:\nMACD\tTENCD\nNCPT\tNghiên cứu phát triển\nQLGD\tQuả

In [16]:
#CREATE DATA PROMPT TEMPLATE
from langchain_core.prompts import ChatPromptTemplate

template = """
You are an expert MySQL developer.

Your task:
- Convert a Vietnamese natural language question into a MySQL SQL query.

STRICT RULES:
- Use ONLY MySQL syntax.
- Use ONLY tables and columns that appear in the provided schema.
- DO NOT guess or invent tables or columns.
- DO NOT explain anything.
- DO NOT wrap the SQL in markdown.
- Output ONLY a single valid SQL query.
- ONLY SELECT queries are allowed (NO INSERT, UPDATE, DELETE, DROP).

DATE & TIME RULES:
- Allowed functions: CURDATE(), NOW(), DATE_SUB(), DATE_ADD(), YEAR(), MONTH(), DAY().
- For phrases like:
  "N ngày gần nhất", "N ngày qua", "trong vòng N ngày":
  ALWAYS use DATE_SUB(CURDATE(), INTERVAL N DAY).
- DO NOT use SQL Server functions (NO GETDATE, NO DATEDIFF, NO DATEPART).

DATABASE SCHEMA:
{schema}

QUESTION:
{question}

SQL QUERY:
"""

prompt = ChatPromptTemplate.from_template(template)


promt=ChatPromptTemplate.from_template(template)

In [17]:
def get_schema():
    schema=db.get_table_info()
    return schema   

In [37]:
import getpass
from langchain_google_genai import ChatGoogleGenerativeAI

api_key = getpass.getpass("Nhập Google API Key: ")

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    google_api_key=api_key
)


In [38]:
def get_schema(db):
    return db.get_table_info()


In [40]:
def generate_sql(question: str):
    schema = get_schema(db)

    prompt_value = prompt.invoke({
        "schema": schema,
        "question": question
    })

    response = llm.invoke(prompt_value)

    sql = response.content.strip()
    return sql


In [41]:
def nl_to_sql_execute(question: str):
    sql = generate_sql(question)

    print("=== GENERATED SQL ===")
    print(sql)

    if not sql.lower().startswith("select"):
        raise ValueError("❌ Only SELECT queries are allowed")

    result = db.run(sql)
    return result


In [42]:
res = nl_to_sql_execute(
    "Cho biết họ tên và mức lương của các giáo viên nữ."
)
print(res)


c:\ProgramData\anaconda3\envs\medicalbot\lib\site-packages\langchain_community\utilities\sql_database.py:338: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "bomon, giaovien", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  metadata_table_names = [tbl.name for tbl in self._metadata.sorted_tables]
c:\ProgramData\anaconda3\envs\medicalbot\lib\site-packages\langchain_community\utilities\sql_database.py:350: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "bomon, giaovien", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  for tbl in self._metadata.sorted_tables


=== GENERATED SQL ===
SELECT HOTEN, LUONG FROM giaovien WHERE PHAI = 'Nữ'
[('Trần Trà Hương', Decimal('2500.00')), ('Nguyễn Ngọc Ánh', Decimal('2200.00')), ('Trần Bạch Tuyết', Decimal('1500.00'))]
